# Analyse von nach Orten benannte Straßen

In [2]:
# Module importieren
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from shapely.geometry import LineString, MultiLineString
import matplotlib.patches as mpatches # Für die Legende
import contextily as ctx

In [ ]:
# CSV-Datei einlesen
file_path_persons = '../../data/clean/Data_Masterarbeit - list_named_after_humans.csv'
file_path_all_streets = '../../data/clean/Data_Masterarbeit - list_complete.csv'
file_path_street_layout = "../../data/raw/lhd_strassenzuege.csv"

df_persons = pd.read_csv(file_path_persons, sep=",")
df_all = pd.read_csv(file_path_all_streets, sep=",")
df_streets = pd.read_csv(file_path_street_layout, sep=";")
df_orte = pd.read_csv("../../data/processed/Geodaten_Orte_Straßennamen.csv", encoding="utf-8")

In [6]:
def kategorisieren_korrigiert(adresse):
    if not isinstance(adresse, str):
        return "unbekannt"
    elif "Dresden" in adresse:
        return "Ortsteil"
    
    # Prüfe ZUERST auf "Sachsen-Anhalt", da dies spezifischer ist.
    elif "Sachsen-Anhalt" in adresse:
        return "Deutschland"  # oder "Sachsen-Anhalt", falls du eine neue Kategorie möchtest
    
    elif "Sachsen" in adresse:
        return "Sachsen"
    # ------------------

    elif "Germany" in adresse or "Deutschland" in adresse:
        return "Deutschland"
    else:
        return "Ausland"

# Wende die korrigierte Funktion an
df_orte['Kategorie'] = df_orte['adresse'].apply(kategorisieren_korrigiert)

## Lage der Orte auf der Karte

In [7]:
# Mittelpunkt Dresden
karte = folium.Map(location=[51.0504, 13.7373], zoom_start=6)

farben = {
    "Ortsteil": "green",
    "Sachsen": "blue",
    "Deutschland": "orange",
    "Ausland": "red",
    "Unbekannt": "gray"
}

for _, row in df_orte.iterrows():
    if pd.notnull(row['lat']) and pd.notnull(row['lon']):
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            color=farben.get(row['Kategorie'], "black"),
            fill=True,
            fill_opacity=0.7,
            popup=f"{row['Ortsname']} ({row['Kategorie']})"
        ).add_to(karte)

karte.save("../../results/maps/Verteilung_Orte_in_Straßennamen.html")

## Verteilung Orte

In [8]:
# Zähle, wie viele Orte es je Kategorie gibt
verteilung = df_orte['Kategorie'].value_counts().sort_values(ascending=True)

# Farben für die einzelnen Kategorien
farben = {
    "Ortsteil": "green",
    "Sachsen": "blue",
    "Deutschland": "orange",
    "Ausland": "red",
    "unbekannt": "gray"
}

#Farben basierend auf den Index-Labels von 'verteilung' zuordnen:
farbliste = [farben.get(k, "#cccccc") for k in verteilung.index]

# Erstelle das Diagramm
plt.figure(figsize=(10, 8))

total = verteilung.values.sum()

# Diagrammaufruf
bars = plt.barh(
    verteilung.index,    # y-Achse (Labels)
    verteilung.values,   # x-Achse (Werte/Breite)
    color=farbliste[:len(verteilung)]
)

labels_for_bars = []
for value in verteilung:
    pct = (value / total) * 100
    labels_for_bars.append(f" {pct:.1f}% ({value})") # Leerzeichen für Abstand

plt.bar_label(
    bars,
    labels=labels_for_bars,
    padding=5,  # Abstand vom Balken
    fontsize=12
)

plt.title(f"Verteilung nach geografischer Lage ")
plt.xlabel("Anzahl") # Achsenbeschriftung hinzufügen
    
# X-Achsen-Limit anpassen, damit Labels Platz haben
plt.xlim(0, verteilung.values.max() * 1.18)

plt.tight_layout() # Wichtig, damit nichts abgeschnitten wird
    
#Diagramm speichern (Dateinamen angepasst)
plt.savefig(f"../../results/charts/bar_charts/Verteilung_geografische_Lage_Orte_von_Straßennamen", dpi=300)
plt.close()


## Verteilung nach Kategorie im Stadtgebiet

### Folium Karte

In [ ]:

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)

strassen_gdf = gpd.GeoDataFrame(
    df_streets.dropna(subset=["geometry"]),
    geometry="geometry",
    crs="EPSG:4326"
)

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# -----------------------------
# 2. Namensdaten laden (CSV mit Benennungstyp)
# -----------------------------
df_all["streetname_clean"] = df_all["Straßenname"].str.strip().str.lower().str.replace(r"\s*\(.*\)", "", regex=True)

# -----------------------------
# 3. Geokodierte Ortsdaten laden + Kategorien
# -----------------------------
orte_kat_df = pd.read_csv("../../data/processed/Geodaten_Orte_Straßennamen.csv", encoding="utf-8")

def kategorisieren(adresse):
    if not isinstance(adresse, str):
        return "unbekannt"
    elif "Dresden" in adresse:
        return "Ortsteil"
    elif "Sachsen" in adresse:
        return "Sachsen"
    elif "Germany" in adresse or "Deutschland" in adresse:
        return "Deutschland"
    else:
        return "Ausland"

orte_kat_df["Ort_Kategorie"] = orte_kat_df["adresse"].apply(kategorisieren)

# -----------------------------
# 4. Nur Straßen nach Orten
# -----------------------------
orte_df = df_all[df_all["Benennungstyp"] == "Ort"].copy()

# Merge mit Orts-Kategorien
orte_df = orte_df.merge(
    orte_kat_df[["Ortsname", "Ort_Kategorie"]],
    left_on="benannt nach",
    right_on="Ortsname",
    how="left"
)

# -----------------------------
# 5. Merge: Lokale Straßendaten ↔ Benennung nach Orten
# -----------------------------
edges_orte = strassen_gdf.merge(
    orte_df[["streetname_clean", "Ort_Kategorie"]],
    left_on="strasse_key",
    right_on="streetname_clean",
    how="inner"
).dropna(subset=["Ort_Kategorie"])

# -----------------------------
# 6. Farben für Kategorien
# -----------------------------
farben = {
    "Ortsteil": "green",
    "Sachsen": "blue",
    "Deutschland": "orange",
    "Ausland": "red",
    "unbekannt": "gray"
}
kategorien = edges_orte["Ort_Kategorie"].unique()

# -----------------------------
# 7. Karte erstellen
# -----------------------------
center = [51.0504, 13.7373]  # Dresden
m = folium.Map(location=center, zoom_start=12)

for kat in kategorien:
    subset = edges_orte[edges_orte["Ort_Kategorie"] == kat]
    color = farben.get(kat, "black")
    fg = folium.FeatureGroup(name=kat, show=True)

    for _, row in subset.iterrows():
        geom = row["geometry"]
        coords = []
        if isinstance(geom, LineString):
            coords = [(pt[1], pt[0]) for pt in geom.coords]
        elif isinstance(geom, MultiLineString):
            for line in geom.geoms:
                coords.extend([(pt[1], pt[0]) for pt in line.coords])
        if coords:
            folium.PolyLine(
                coords, color=color, weight=3, opacity=0.8,
                tooltip=row["strasse"]
            ).add_to(fg)
    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


m.save("../../results/maps/Verteilung_Orte_in_Stadt.html")

### Statisches PNG

In [ ]:
# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)

strassen_gdf = gpd.GeoDataFrame(
    df_streets.dropna(subset=["geometry"]),
    geometry="geometry",
    crs="EPSG:4326"
)

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Namensdaten laden (CSV mit Benennungstyp)
df_all["streetname_clean"] = df_all["Straßenname"].str.strip().str.lower().str.replace(r"\s*\(.*\)", "", regex=True)


# 3. Geokodierte Ortsdaten laden + Kategorien
df_orte = pd.read_csv("../../data/processed/Geodaten_Orte_Straßennamen.csv", encoding="utf-8")

def kategorisieren(adresse):
    if not isinstance(adresse, str):
        return "unbekannt"
    elif "Dresden" in adresse:
        return "Ortsteil"
    elif "Sachsen" in adresse:
        return "Sachsen"
    elif "Germany" in adresse or "Deutschland" in adresse:
        return "Deutschland"
    else:
        return "Ausland"

df_orte["Ort_Kategorie"] = df_orte["adresse"].apply(kategorisieren)

# 4. Nur Straßen nach Orten
df_location = df_all[df_all["Benennungstyp"] == "Ort"].copy()

# Merge mit Orts-Kategorien
df_location = df_location.merge(
    df_orte[["Ortsname", "Ort_Kategorie"]],
    left_on="benannt nach",
    right_on="Ortsname",
    how="left"
)

# 5. Merge: Lokale Straßendaten ↔ Benennung nach Orten
edges_orte = strassen_gdf.merge(
    df_location[["streetname_clean", "Ort_Kategorie"]],
    left_on="strasse_key",
    right_on="streetname_clean",
    how="inner"
).dropna(subset=["Ort_Kategorie"])

# 6. Farben für Kategorien
farben = {
    "Ortsteil": "green",
    "Sachsen": "blue",
    "Deutschland": "orange",
    "Ausland": "red",
    "unbekannt": "gray"
}
# Kategorien, die tatsächlich in den Daten vorkommen
kategorien = edges_orte["Ort_Kategorie"].unique()

# Füge eine Farbspalte zum GeoDataFrame hinzu
edges_orte["plot_color"] = edges_orte["Ort_Kategorie"].map(farben)


# 7. Statische Karte (PNG) erstellen
print("Erstelle statische PNG-Karte...")

# WICHTIG: contextily (Basiskarten) benötigt das Web-Mercator-Format (EPSG:3857)
edges_orte_web_mercator = edges_orte.to_crs(epsg=3857)

# Erstelle eine Matplotlib-Figur und -Achse
fig, ax = plt.subplots(figsize=(14, 14))

# Zeichne die Straßen auf die Achse
edges_orte_web_mercator.plot(
    ax=ax,
    color=edges_orte_web_mercator["plot_color"], # Verwende die vordefinierte Farbspalte
    linewidth=1.5,  # Linienstärke anpassen
    alpha=0.7
)

# Füge eine Basiskarte hinzu (z.B. CartoDB Positron, die ist schön dezent)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=13)

# Achsenbeschriftungen ausblenden
ax.axis('off')


# 8. Legende für PNG erstellen

# Erstelle "Patches" (Platzhalter) für die Legende
patches = []
# Sicherstellen, dass die Legende in der richtigen Reihenfolge ist (optional)
gefundene_kategorien = sorted(
    [kat for kat in farben if kat in kategorien], 
    key=lambda x: list(farben.keys()).index(x)
)

for kat in gefundene_kategorien:
    patch = mpatches.Patch(color=farben[kat], label=kat)
    patches.append(patch)

# Füge die Legende zur Karte hinzu
ax.legend(
    handles=patches, 
    loc='lower right',
    fontsize='large',
    title_fontsize='x-large',
    frameon=True,
    facecolor='white',
    framealpha=0.8
)

# 9. Karte als PNG speichern
output_path = "../../results/images/Verteilung_Orte_in_Stadt.png"
plt.title(f"Lage von nach Orten benannte Straßen", fontsize=16)
plt.savefig(
    output_path, 
    dpi=300,
    bbox_inches='tight' 
)
plt.close()

Erstelle statische PNG-Karte...
